In [254]:
def get_user_profile_from():
    skills = [
        'Agentic AI', 
        'Retrieval Augmented Generation (RAG)', 
        'Large Language Models (LLMs)', 
        'LangChain', 
        'Django',
        'LSTM',
        'Flask',
        'Transformers',
        'RNN',
        'TensorFlow',
        'Supervised Learning',
        'Unsupervised Learning',
        'Pytorch',
        'CNN',
        'DBMS',
        'DSA',
        'Statistics',
        'SQL',
        'AI',
        'Linear Algebra',
        'Calculus',
        'PCA',
        'Python',
        'Seaborn',
        'Pandas',
        'Numpy',
        'Matplotlib',
        'Scikit-learn',
        'Data Analytics',
        'Data Science',
        'Machine Learning',
        'Jupyter Notebook',
        'Git'
    ]
    certifications = [
        {
            'name' : 'Google Advanced Data Analytics',
            'organization' : 'Google'
        },
        {
            'name' : 'IBM Data Science Professional Certificate',
            'organization' : 'IBM'
        }, 
        {
            'name' : 'IBM RAG and Agentic AI: Build Next-Gen AI Systems',
            'organization' : 'IBM'
        },
        {
            'name' : 'Deep Learning',
            'organization' : 'DeepLearningAI'
        },
        {
            'name' : 'Machine Learning',
            'organization' : 'DeepLearningAI'
        },
        {
            'name' : 'Mathematics for Machine Learning Specialization',
            'organization' : 'Imperial College London'
        },
        {
            'name' : 'Mathematics for Machine Learning and Data Science',
            'organization' : 'DeepLearning.AI'
        },
        {
            'name' : 'Python for Everybody Specialization',
            'organization' : 'University of Michigan'
        }
    ]
    
    projects = [
        {
            'name' : 'BiasBusterAI: Text Bias Detection System',
            'start_date' : 'October 2025',
            'end_date' : 'October 2025',
            'description' : 'Developed AI web app detecting biases (race, gender, profession, religion) with Bidirectional LSTM with Self Attention, achieving ~98% accuracy via TensorFlow. Created Flask interface with Plotly for real-time visualization of bias probabilities and attention weights. https://github.com/sheb1lmsp/BiasBusterAI'
        },
        {
            'name' : 'SmartAttend: An Automated Attendance Management System',
            'start_date' : 'May 2025',
            'start_date' : 'August 2025',
            'description' : 'Developed a smart attendance management system using facial recognition to automate classroom attendance tracking. Leveraged PyTorch, MTCNN, and InceptionResNetV1 to build and train machine learning models for accurate face detection and recognition. Integrated the ML pipeline with a Django web application, featuring role-based dashboards for admins, teachers, and students. Enabled efficient attendance marking via group photos, streamlining academic workflows. https://github.com/sheb1lmsp/smart_attend'
        }
    ]
    
    education = [
        {
            'degree' : 'Bachelor of Computer Applications',
            'university' : 'Bangalore University',
            'start_date' : 'August 2022',
            'end_date' : 'June 2025',
            'cgpa' : '8.82'
        },
    ]
    
    experience = [
        {
            'title' : '',
            'employment_type' : '',
            'company' : '',
            'start_date' : '',
            'end_date' : '',
            'location' : '',
            'description' : ''
        }
    ]

    return {'education' : education, 'certifications' : certifications, 'projects' : projects, 'skills' : skills, 'experience' : experience}

In [255]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

llm = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash')

In [266]:
from typing import TypedDict, Literal, List, Dict, Optional, Any

class Project(TypedDict):
    name: str
    start_date: str
    end_date: str
    description: str

class Certification(TypedDict):
    name: str
    organization: str

class Education(TypedDict):
    degree: str
    university: str
    start_date: str
    end_date: str
    cgpa: float

class Experience(TypedDict):
    title: str
    employment_type: str
    company: str
    start_date: str
    end_date: str
    location: str
    description: str

class State(TypedDict):
    input_text: str
    agent_action: Optional[
        Literal[
            "course_recommender",
            "project_recommender",
            "interview_coach",
            "learning_path_advisor",
            "resume_builder",
            "skill_analyzer",
        ]
    ]

    skills: Optional[List[str]]
    certifications: Optional[List[Certification]]
    projects: Optional[List[Project]]
    education: Optional[List[Education]]
    experience: Optional[List[Experience]]

    response: Optional[str]
    final_output: Optional[str]

    metadata: Optional[Dict]
    resume_text: Optional[str]

In [267]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class AgentType(BaseModel):
    agent_name: str = Field(description="The name of the agent that should handle this query")

def router(state: State) -> State:
    """Router function for CareerGraph AI — decides which agent to use based on input_text."""
    
    router_prompt = ChatPromptTemplate.from_messages(
       [(
           "system",
           """
           You are the RouterAgent for CareerGraph AI — an intelligent, LLM-powered career assistant.
           
           Your task:
           - Read the user's query carefully.
           - Determine which specialized agent should handle it.
           
           Return 'only one' of the following agent names as a 'plain string' (no punctuation, no explanation):
           
           1. "skill_analyzer" → When the user talks about skills, strengths, weaknesses, or what they’re good at.
           2. "career_advisor" → When the user asks about job roles, next career moves, or suitable positions.
           3. "project_recommender" → When the user asks for project ideas, feedback, or inspiration for new projects.
           4. "course_recommender" → When the user asks for learning resources, courses, or certifications.
           5. "learning_path_advisor" → When the user wants a structured roadmap or skill-learning order.
           6. "resume_builder" → When the user wants to create, optimize, or improve their resume.
           7. "interview_coach" → When the user wants interview preparation, practice questions, or feedback.
           
           Output format:
           Return 'only' the agent name as a raw string like: skill_analyzer
           """
       ),
        ("human","User query: {input_text}")
       ]
    )

    chain = router_prompt | llm.with_structured_output(AgentType)

    response = chain.invoke({'input_text' : state['input_text']})

    return {**state, 'agent_action' : response.agent_name}
    

In [268]:
from pprint import pprint
result = router({'input_text' : "what is my weakness"})
pprint(result)

{'agent_action': 'skill_analyzer', 'input_text': 'what is my weakness'}


In [232]:
def get_user_profile(state: State) -> State:
    """
    Node that loads the user profile from the database and updates the graph state.
    """
    
    profile_data = get_user_profile_from()

    state["skills"] = profile_data.get("skills", [])
    state["education"] = profile_data.get("education", [])
    state["experience"] = profile_data.get("experience", [])
    state["projects"] = profile_data.get("projects", [])
    state["certifications"] = profile_data.get("certifications", [])

    return state

In [233]:
result = get_user_profile(result)
pprint(result)

{'agent_action': 'skill_analyzer',
 'certifications': [{'name': 'Google Advanced Data Analytics',
                     'organization': 'Google'},
                    {'name': 'IBM Data Science Professional Certificate',
                     'organization': 'IBM'},
                    {'name': 'IBM RAG and Agentic AI: Build Next-Gen AI '
                             'Systems',
                     'organization': 'IBM'},
                    {'name': 'Deep Learning', 'organization': 'DeepLearningAI'},
                    {'name': 'Machine Learning',
                     'organization': 'DeepLearningAI'},
                    {'name': 'Mathematics for Machine Learning Specialization',
                     'organization': 'Imperial College London'},
                    {'name': 'Mathematics for Machine Learning and Data '
                             'Science',
                     'organization': 'DeepLearning.AI'},
                    {'name': 'Python for Everybody Specialization',
      

In [153]:
def course_recommender(state: State) -> State:
    """
    Suggests relevant courses or certifications based on user's profile,
    while avoiding those the user has already completed or mentioned.
    """

    user_skills = state.get("skills", "None")
    education = state.get("education", "None")
    projects = state.get("projects", "None")
    certifications = state.get("certifications", "None")
    user_query = state.get("input_text", "")

    course_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            You are the CourseRecommender Agent for CareerGraph AI.

            Your goal is to suggest highly relevant 'online courses or certifications'
            from popular learning platforms (Coursera, Udemy, edX, Google, LinkedIn Learning, etc.)
            that can help the user grow in their desired career direction.

            ⚠️ IMPORTANT RULES:
            - DO NOT recommend any course, topic, or certification the user already has or mentioned.
            - DO NOT suggest duplicates of known certifications.
            - Focus on 'next-step' or 'complementary learning' opportunities.
            - Recommend a maximum of 3–5 courses.
            - For each course: include its platform and a one-line reason why it's relevant.
            - Output must be 'plain text' (no JSON, no markdown).
            """
        ),
        (
            "human",
            """
            User query or goal:
            {user_query}

            User profile details:
            - Skills: {user_skills}
            - Education: {education}
            - Projects: {projects}
            - Completed Certifications: {certifications}

            Return 3–5 courses the user has 'not yet done' that could strengthen or extend their abilities.

            Format strictly like this:
            1. [Course Name] — [Platform]
               Why: [One-line reason]
            2. [Course Name] — [Platform]
               Why: [One-line reason]
            """
        )
    ])

    chain = course_prompt | llm

    response = chain.invoke({
        "user_query": user_query,
        "user_skills": user_skills,
        "education": education,
        "projects": projects,
        "certifications": certifications,
    })

    state["response"] = response.content.strip()
    state["final_output"] = state["response"]

    return state

In [137]:
result = course_recommender(result)
print(result['final_output'])

1. Machine Learning Engineering for Production (MLOps) — DeepLearning.AI (Coursera)
   Why: This course will equip you with the skills to deploy, monitor, and maintain your advanced ML models in real-world production environments.
2. Reinforcement Learning Specialization — University of Alberta (Coursera)
   Why: Expand your AI expertise by delving into a new paradigm of machine learning focused on decision-making and optimal control in dynamic environments.
3. Google Cloud Professional Machine Learning Engineer Professional Certificate — Google (Coursera)
   Why: Gain hands-on experience and certification in designing, building, and deploying ML solutions on a leading cloud platform.
4. Data Engineering with Google Cloud Professional Certificate — Google (Coursera)
   Why: Strengthen your ability to build and manage robust data pipelines and infrastructure, essential for supporting large-scale machine learning projects.


In [154]:
def project_recommender(state: State) -> State:
    """
    Suggests creative and technically relevant project ideas for the user,
    avoiding overlap with existing projects.
    """

    user_skills = ", ".join(state.get("skills", "None"))
    education = state.get("education", "None")
    experience = state.get("experience", "None")
    projects = state.get("projects", "None")
    certifications = state.get("certifications", "None")
    user_query = state.get("input_text", "")

    project_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            You are the ProjectRecommender Agent for CareerGraph AI.

            Your task:
            - Suggest 'unique, high-impact project ideas' tailored to the user’s skills, experience, and goals.
            - The projects should help the user 'build portfolio depth' and 'improve employability'.

            ⚙️ Rules:
            - DO NOT suggest projects that are too similar to the user’s existing ones.
            - Suggest 3–5 'new and creative' project ideas.
            - Include a one-line reason why each project is valuable.
            - Prefer ideas that are relevant to the user’s current skills but slightly challenging to help them grow.
            - Avoid trivial projects (e.g., “To-Do App”, “Calculator App”).
            - Output must be 'plain text only' (no markdown, no JSON).

            🎯 Example areas:
            - If the user has AI/ML skills → suggest applied ML, NLP, GenAI, or automation projects.
            - If the user is from software/dev background → suggest scalable systems or automation tools.
            - If the user is from data analytics → suggest visualization dashboards or predictive models.
            """
        ),
        (
            "human",
            """
            User query or career goal:
            {user_query}

            User profile:
            - Skills: {user_skills}
            - Education: {education}
            - Experience: {experience}
            - Certifications: {certifications}
            - Existing Projects: {projects}

            Recommend 3–5 unique project ideas the user can build next.
            Each idea should include:
            1. Project Name — short and creative
            2. One-sentence description
            3. Why it's valuable

            Output format:
            1. [Project Name]
               Description: ...
               Why: ...
            """
        )
    ])

    chain = project_prompt | llm

    response = chain.invoke({
        "user_query": user_query,
        "user_skills": user_skills,
        "education": education,
        "experience": experience,
        "projects": projects,
        "certifications" : certifications,
    })

    state["response"] = response.content.strip()
    state["final_output"] = state["response"]

    return state


In [155]:
result = project_recommender(result)
print(result['final_output'])

1. DocuAgent: Intelligent Knowledge Retrieval System
   Description: Develop an AI agent that uses Retrieval Augmented Generation (RAG) and LangChain to provide accurate, context-aware answers from a large, domain-specific document corpus, enhancing information accessibility and reducing hallucination.
   Why: Directly applies advanced LLM techniques (RAG, Agentic AI, LangChain) to a practical problem, demonstrating ability to build robust, verifiable AI systems.

2. CodeCraft AI: Contextual Code Generation Assistant
   Description: Build an AI assistant that generates code snippets or full functions based on natural language prompts, leveraging fine-tuned Large Language Models and a retrieval mechanism to incorporate best practices or existing code patterns.
   Why: Showcases advanced generative AI application, understanding of code semantics, and ability to integrate LLMs for practical developer tools.

3. SentinelStream: Real-time Anomaly Detection System
   Description: Develop a s

In [217]:
class LearningPath(BaseModel):
    target_role: str = Field(description="The goal or role the user wants to prepare for")
    required_skills: List[str] = Field(description="Skills or topics the user needs to learn next")
    roadmap_steps: List[str] = Field(description="Chronological learning steps")
    recommended_resources: List[str] = Field(description="Optional recommended learning resources")
    summary: str = Field(description="A short summary of the learning plan")

def learning_path_advisor(state: State) -> State:
    """
    Node that generates a step-by-step learning roadmap for the user's desired goal.
    Uses known skills and background to avoid redundancy.
    """

    user_input = state.get("input_text", "")
    known_skills = " ".join(state.get("skills", "None"))
    certifications = state.get("certifications", "None")
    education = state.get("education", "None")
    experience = state.get("experience", "None")
    projects = state.get("projects", "None")

    learning_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            You are the LearningPath Advisor for CareerGraph AI.
            You specialize in creating personalized, practical, and efficient learning roadmaps 
            based on the user’s profile, skills, and goals.

            Guidelines:
            - Recommend skills and learning steps relevant to the user's career goal.
            - DO NOT include skills, courses, or certifications the user already has.
            - Keep the roadmap structured, progressive, and goal-oriented.
            - Assume the user prefers real-world applicability (projects, portfolios, and measurable milestones).
            - If no clear goal is stated, infer one from their background and interests.
            """
        ),
        (
            "human",
            """
            User Query: {user_input}

            Known Skills: {known_skills}
            Certifications: {certifications}
            Education: {education}
            Experience: {experience}
            Projects: {projects}

            Provide:
            - target_role
            - required_skills (excluding known ones)
            - roadmap_steps (chronological)
            - recommended_resources (optional)
            - summary
            """
        )
    ])

    chain = learning_prompt | llm.with_structured_output(LearningPath)

    response = chain.invoke({
        "user_input": user_input,
        "known_skills": known_skills,
        "certifications": certifications,
        "education": education,
        "experience": experience,
        "projects": projects,
    })

    formatted_output = (
        f"🎯 **Target Role:** {response.target_role}\n\n"
        f"🧩 **Required Skills:** {', '.join(response.required_skills)}\n\n"
        f"🪜 **Learning Roadmap:**\n" + "\n".join([f"- {step}" for step in response.roadmap_steps]) + "\n\n"
        f"📘 **Recommended Resources:**\n" + "\n".join([f"- {r}" for r in response.recommended_resources]) + "\n\n"
        f"📝 **Summary:** {response.summary}"
    )

    state["response"] = formatted_output
    state["final_output"] = formatted_output

    return state


In [225]:
result = learning_path_advisor(result)
print(result['final_output'])

🎯 **Target Role:** AI Engineer (LLM Focus)

🧩 **Required Skills:** Advanced Prompt Engineering, LLM Fine-tuning (LoRA, QLoRA), Vector Database Implementation (Pinecone, Weaviate, Milvus), MLOps for LLMs (Docker, Kubernetes, Cloud ML Platforms like AWS Sagemaker/GCP Vertex AI), Reinforcement Learning from Human Feedback (RLHF), LLM Evaluation and Benchmarking, System Design for Scalable LLM Applications, LLM Optimization Techniques (Quantization, Pruning)

🪜 **Learning Roadmap:**
- Master Advanced Prompt Engineering: Learn and apply advanced techniques (e.g., Chain-of-Thought, Tree-of-Thought, Self-Consistency) to build sophisticated LLM interactions. Milestone: Develop a project demonstrating a multi-turn, context-aware LLM agent using advanced prompting.
- Deep Dive into LLM Fine-tuning: Understand and implement parameter-efficient fine-tuning (PEFT) methods like LoRA and QLoRA on open-source LLMs. Milestone: Fine-tune a specialized LLM for a specific domain task and evaluate its perf

In [234]:
def skill_analyzer(state: State) -> State:
    """
    Analyzes the user's skills, projects, and experience, and returns 
    an unstructured natural-language summary of their strengths and upskilling recommendations.
    """

    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            You are CareerGraph AI’s Skill Analyzer Agent.
            Your job:
            - Analyze the user's current skills, experience, and projects.
            - Identify their strongest skills and areas of expertise.
            - Point out gaps or weaknesses.
            - Suggest a few new skills, courses, or technologies to improve their profile.
            - Be concise and specific, using bullet points or a short paragraph.

            Output format:
            - Core Strengths:
            - Missing / Weak Skills:
            - Recommended Upskilling:
            - Suggested Courses / Certifications:
            """
        ),
        (
            "human",
            """
            User Query: {input_text}

            Current Skills: {skills}
            Education: {education}
            Experience: {experience}
            Projects: {projects}
            Certifications: {certifications}
            """
        )
    ])

    chain = prompt | llm

    response = chain.invoke({
        "input_text": state.get("input_text", ""),
        "skills": state.get("skills", []),
        "education": state.get("education", []),
        "experience": state.get("experience", []),
        "projects": state.get("projects", []),
        "certifications": state.get("certifications", [])
    })

    state["response"] = response.content
    state["final_output"] = response.content

    return state


In [236]:
result = skill_analyzer(result)
print(result['final_output'])

Based on your impressive skills, projects, and certifications, you have a very strong foundation in AI, Machine Learning, Deep Learning, and web development. Your projects demonstrate practical application of advanced concepts like RAG, Agentic AI, LLMs, NLP, and Computer Vision.

However, to address your query about weaknesses and further enhance your profile, here are some areas:

*   **Core Strengths:**
    *   **Deep Learning & NLP Expertise:** Strong grasp of LLMs, RAG, Agentic AI, Transformers, LSTM, CNN, RNN, and practical application in projects (BiasBusterAI, SmartAttend).
    *   **Solid ML & Data Science Foundation:** Comprehensive skills in supervised/unsupervised learning, Python data stack (Pandas, Numpy, Scikit-learn), statistics, and mathematical foundations.
    *   **Web Integration:** Ability to integrate ML models into functional web applications using Flask and Django.
    *   **Robust Certifications:** Validated knowledge across core ML, DL, Data Science, and adva

In [203]:
class JobDescriptionModel(BaseModel):
    is_job_description: bool = Field(description="True if the input contains a job description or job post.")
    job_title: str = Field(description="The job title or role name, if mentioned.", default="")
    company: str = Field(description="The company name, if mentioned.", default="")
    required_skills: list[str] = Field(description="List of technical or soft skills mentioned.", default=[])
    responsibilities: list[str] = Field(description="List of key responsibilities or tasks mentioned.", default=[])
    experience_level: str = Field(description="Experience level (e.g., Entry-level, Mid-level, Senior), if detectable.", default="")
    summary: str = Field(description="2-line summary of what the role is about.", default="")

def job_description_parser(state: State) -> State:
    """
    Parses job descriptions from user input and stores structured details
    in state['metadata']['job_description'].
    """
    user_query = state.get("input_text", "")

    jd_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            You are the JobDescriptionParser Agent for CareerGraph AI.
            Your job:
            - Detect if the user provided a job description.
            - If yes, extract structured details:
              • job_title
              • company
              • required_skills
              • responsibilities
              • experience_level
              • summary
            - If not, set is_job_description=False.
            - Be factual and concise.
            """
        ),
        ("human", "User input:\n{user_query}")
    ])

    chain = jd_prompt | llm.with_structured_output(JobDescriptionModel)
    response = chain.invoke({"user_query": user_query})

    if state.get("metadata") is None:
        state["metadata"] = {}


    state["metadata"]["job_description"] = response.model_dump()

    return state


In [206]:
result = job_description_parser(result)
result['metadata']['job_description']

{'is_job_description': True,
 'job_title': 'Data Scientist',
 'company': 'Tesco',
 'required_skills': ['Machine Learning',
  'Data Mining',
  'Advanced Algorithms',
  'Statistics',
  'Python',
  'Predictive Modelling',
  'Operational Research',
  'Deep Learning',
  'Time Series Modelling',
  'Hadoop',
  'Spark',
  'Mathematical Modeling',
  'Communication',
  'Problem Solving'],
 'responsibilities': ['Model complex business problems and deploy data products at scale',
  'Understand difficult business problems and prototype solutions with minimal support',
  'Apply, modify and design algorithms and mathematical models to solve business problems on top of big data architectures (Hadoop, Spark)',
  'Validate, document and present the modeling process and performances',
  'Communicate complex solutions in a clear, understandable way to non-experts',
  'Promote data science across Tesco and promote Tesco across the external Data Science community'],
 'experience_level': 'Senior',
 'summary'

In [240]:
import fitz  
from docx import Document

def extract_text_from_pdf(file_path: str) -> str:
    """Extract text from PDF using PyMuPDF."""
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text") + "\n"
    return text.strip()


def extract_text_from_docx(file_path: str) -> str:
    """Extract text from DOCX using python-docx."""
    doc = Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text.strip()


def extract_resume_text(file_path: str) -> str:
    """
    Unified loader that detects file type (.pdf or .docx)
    and extracts clean text for LLM input.
    """
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please upload a PDF or DOCX resume.")


In [269]:
class ResumeModel(BaseModel):
    name: str = Field(description="Full name of the candidate, if available", default="")
    email: str = Field(description="Email address of the candidate", default="")
    phone: str = Field(description="Phone number, if mentioned", default="")
    summary: str = Field(description="Brief 2-3 line professional summary from the resume", default="")
    skills: List[str] = Field(description="List of technical or soft skills", default=[])
    education: List[str] = Field(description="List of educational qualifications and universities", default=[])
    experience: List[str] = Field(description="List of work experience entries or job roles", default=[])
    certifications: List[str] = Field(description="List of certifications or achievements", default=[])
    projects: List[str] = Field(description="List of project titles or descriptions", default=[])
    total_experience_years: float = Field(description="Approximate total years of experience", default=0.0)

def resume_parser(state: State) -> State:
    """
    Parses resume text from user input and stores structured details
    inside state['metadata']['resume_data'].
    Does NOT overwrite or modify top-level state fields.
    """

    resume_text = state.get('resume_text', "None")

    resume_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            You are the ResumeParser Agent for CareerGraph AI.
            Your job:
            - Analyze the provided text (which may come from a resume).
            - Extract the following structured fields:
              • name
              • email
              • phone
              • summary
              • skills
              • education
              • experience
              • certifications
              • projects
              • total_experience_years (approx)
            - If no resume details are detected, leave fields empty.
            - Return data in structured JSON matching the ResumeModel schema.
            """
        ),
        ("human", "Resume text:\n{resume_text}")
    ])

    chain = resume_prompt | llm.with_structured_output(ResumeModel)
    response = chain.invoke({"resume_text": resume_text})

    if state.get("metadata") is None:
        state["metadata"] = {}

    state["metadata"]["resume_data"] = response.model_dump()

    return state


In [270]:
file_path = "../../../../Desktop/CV/Mohammed_Shebil_Resume.pdf"

result = resume_parser({'resume_text' : extract_resume_text(file_path)})

In [271]:
result['metadata']

{'resume_data': {'name': 'Mohammed Shebil P',
  'email': 'shebil.mohammed.p@gmail.com',
  'phone': '+91 86062 19789',
  'summary': 'Recent graduate with strong skills in Machine Learning, AI, and data analytics, seeking opportunities to apply technical expertise in real-world projects.',
  'skills': ['Supervised Learning',
   'Unsupervised Learning',
   'NLP',
   'Computer Vision',
   'CNN',
   'RNN',
   'Transformers',
   'LLMs',
   'Generative AI',
   'Python (Advanced)',
   'SQL (Intermediate)',
   'PyTorch',
   'TensorFlow',
   'Keras',
   'Scikit-learn',
   'XGBoost',
   'Pandas',
   'NumPy',
   'PySpark',
   'Django',
   'Flask',
   'SQLite',
   'Git',
   'Google Cloud Platform (Vertex AI, BigQuery)',
   'Seaborn',
   'Matplotlib',
   'Tableau',
   'Power BI'],
  'education': ['Bachelor of Computer Applications - Bangalore University, Bengaluru (August 2022 - June 2025)'],
  'experience': [],
  'certifications': ['Deep Learning Specialization, DeepLearning.AI, 2025',
   'Machine 

In [273]:
def interview_coach(state: State) -> State:
    """
    CareerGraph AI - Interview Coach Agent
    Provides personalized interview guidance based on job description, resume, or user data.
    """
    job_description = None
    resume_data = None

    if state.get("metadata"):
        job_description = state["metadata"].get("job_description")
        resume_data = state["metadata"].get("resume_data")

    skills = state.get("skills", [])
    experience = state.get("experience", [])
    education = state.get("education", [])
    projects = state.get("projects", [])
    certifications = state.get("certifications", [])

    context_parts = []

    if job_description and job_description.get("is_job_description"):
        context_parts.append(
            f"Job Role: {job_description.get('job_title', 'N/A')}\n"
            f"Company: {job_description.get('company', 'N/A')}\n"
            f"Required Skills: {', '.join(job_description.get('required_skills', []))}\n"
            f"Responsibilities: {', '.join(job_description.get('responsibilities', []))}\n"
            f"Experience Level: {job_description.get('experience_level', 'N/A')}\n"
            f"Summary: {job_description.get('summary', '')}\n"
        )

    if resume_data:
        context_parts.append(
            "Candidate Resume:\n"
            f"Name: {resume_data.get('name', 'N/A')}\n"
            f"Skills: {', '.join(resume_data.get('skills', []))}\n"
            f"Experience: {', '.join(resume_data.get('experience', []))}\n"
            f"Projects: {', '.join(resume_data.get('projects', []))}\n"
            f"Education: {', '.join(resume_data.get('education', []))}\n"
        )

    if not context_parts:
        context_parts.append(
            "User profile data:\n"
            f"Skills: {', '.join(skills)}\n"
            f"Experience: {', '.join([exp['title'] + ' at ' + exp['company'] for exp in experience])}\n"
            f"Education: {', '.join([edu['degree'] + ' at ' + edu['university'] for edu in education])}\n"
            f"Projects: {', '.join([p['name'] for p in projects])}\n"
        )

    combined_context = "\n\n".join(context_parts)

    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            You are the Interview Coach Agent for CareerGraph AI.
            Your goal:
            - Prepare the user for their upcoming job interview.
            - Analyze the given job description, resume, or available data.
            - Identify the key technical and behavioral areas the user should prepare for.
            - Suggest possible interview questions and tips for answering them effectively.
            - If the role or company is mentioned, tailor your advice to that.

            Output structure:
            1. Role Context (1–2 lines)
            2. Key Focus Areas
            3. Likely Technical Questions
            4. Behavioral Questions
            5. Preparation Tips
            6. Bonus Recommendations (optional)
            """
        ),
        (
            "human",
            """
            User Query: {input_text}

            Context Information:
            {combined_context}
            """
        )
    ])

    chain = prompt | llm

    response = chain.invoke({
        "input_text": state.get("input_text", ""),
        "combined_context": combined_context,
    })

    state["response"] = response.content
    state["final_output"] = response.content

    return state


In [275]:
def resume_builder(state: State) -> State:
    """
    CareerGraph AI - Resume Builder Agent (ATS + HR Selection Model)

    Phase 1: ATS Engine creates 5 optimized resume variations for the target role.
    Phase 2: HR Reviewer selects the best one based on impact, clarity, and relevance.
    """
    job_description = None
    resume_data = None

    if state.get("metadata"):
        job_description = state["metadata"].get("job_description")
        resume_data = state["metadata"].get("resume_data")

    skills = state.get("skills", [])
    experience = state.get("experience", [])
    education = state.get("education", [])
    projects = state.get("projects", [])
    certifications = state.get("certifications", [])

    context_parts = []

    if job_description and job_description.get("is_job_description"):
        context_parts.append(
            f"Job Role: {job_description.get('job_title', 'N/A')}\n"
            f"Company: {job_description.get('company', 'N/A')}\n"
            f"Required Skills: {', '.join(job_description.get('required_skills', []))}\n"
            f"Responsibilities: {', '.join(job_description.get('responsibilities', []))}\n"
            f"Experience Level: {job_description.get('experience_level', 'N/A')}\n"
            f"Summary: {job_description.get('summary', '')}\n"
        )

    if resume_data:
        context_parts.append(
            "Existing Resume:\n"
            f"Name: {resume_data.get('name', 'N/A')}\n"
            f"Skills: {', '.join(resume_data.get('skills', []))}\n"
            f"Experience: {', '.join(resume_data.get('experience', []))}\n"
            f"Projects: {', '.join(resume_data.get('projects', []))}\n"
            f"Education: {', '.join(resume_data.get('education', []))}\n"
        )

    if not context_parts:
        context_parts.append(
            "User profile data:\n"
            f"Skills: {', '.join(skills)}\n"
            f"Experience: {', '.join([exp['title'] + ' at ' + exp['company'] for exp in experience])}\n"
            f"Education: {', '.join([edu['degree'] + ' at ' + edu['university'] for edu in education])}\n"
            f"Projects: {', '.join([p['name'] for p in projects])}\n"
            f"Certifications: {', '.join([c['name'] for c in certifications])}\n"
        )

    combined_context = "\n\n".join(context_parts)

    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            You are the Resume Builder Agent for CareerGraph AI.
            Think in two roles:

            Phase 1 — ATS Engine:
            • Imagine you are an Applicant Tracking System (ATS) analyzing a candidate profile and job description.
            • Generate FIVE unique, highly ATS-friendly resumes.
            • Each resume should follow this consistent format:
                ======================
                [FULL NAME]
                [PROFESSIONAL SUMMARY]
                [SKILLS]
                [EXPERIENCE]
                [PROJECTS]
                [EDUCATION]
                [CERTIFICATIONS] (optional)
                ======================
            • Focus on keyword density, clear headers, measurable results, and job relevance.
            • Each resume should slightly vary in focus or phrasing — one more technical, one more business-focused, one more concise, etc.

            Phase 2 — HR Reviewer:
            • Now imagine you are a human recruiter reviewing those 5 resumes.
            • Choose the ONE resume that would most likely get shortlisted for an interview.
            • Explain briefly (2–3 lines) why this version stands out.
            • Output only the chosen resume and the short explanation at the end.

            Output format:
            ======================
            [FINAL SELECTED RESUME TEXT]
            ======================
            HR Comment: <why it was chosen>
            """
        ),
        (
            "human",
            """
            User Query: {input_text}

            Context Information:
            {combined_context}
            """
        )
    ])

    chain = prompt | llm

    response = chain.invoke({
        "input_text": state.get("input_text", ""),
        "combined_context": combined_context,
    })

    state["response"] = response.content
    state["final_output"] = response.content

    return state
